In [1]:
def CountDown(n):
    while n > 0:
        n -= 1

%time CountDown(100000000)

CPU times: user 7.24 s, sys: 46.4 ms, total: 7.28 s
Wall time: 7.53 s


In [3]:
from threading import Thread

def multi_thread_process():
    n = 100000000

    t1 = Thread(target=CountDown, args=[n // 4])
    t2 = Thread(target=CountDown, args=[n // 4])
    t3 = Thread(target=CountDown, args=[n // 4])
    t4 = Thread(target=CountDown, args=[n // 4])
    t1.start()
    t2.start()
    t3.start()
    t4.start()
    t1.join()
    t2.join()
    t3.join()
    t4.join()

%time multi_thread_process()

CPU times: user 6.84 s, sys: 86.8 ms, total: 6.92 s
Wall time: 6.92 s


In [4]:
import sys

a = []
b = a
sys.getrefcount(a)

3

CPython引进GIL其实主要是这么两个原因：

* 一是设计者为了规避类似于内存管理这样的复杂的竞争风险问题（race condition）；
* 二是因为CPython大量使用C语言库，但大部分C语言库都不是原生线程安全的（线程安全会降低性
能和增加复杂度）

**Python 的线程安全**

In [9]:
import threading

n = 0

def foo():
    global n
    n += 1

threads = []

for i in range(100):
    t = threading.Thread(target=foo)
    threads.append(t)

for thread in threads:
    thread.start()

for t in threads:
    t.join()

print(n)

100


所以，千万别想着，有了GIL你的程序就可以高枕无忧了，我们仍需要去注意线程安全。正如我开头所说，**GIL的设计，主要是为了方便CPython解释器层面的编写者，而不是Python应用层面的程序员。**作为Python的使用者，我们还是需要lock等工具，用来确保线程安全；

In [10]:
import threading

n = 0
lock = threading.Lock()

def foo():
    global n
    with lock:
        n += 1

**如何绕过GIL**

总的来说，你只需要重点记住，绕过GIL的大致思路有这么两种就够了：

1. 绕过CPython，使用JPython（Java实现的Python解释器）等别的实现；
2. 把关键性能代码，放到逼得语言（一般是C++）中实现。